# Dynamic Programming Exercises

## Exercise 1


### 1. State variables:

Stock of oil $s_t$ and price $p_t$.

### 2. Control variables:

Amount of oil to sell $x_t$.

### 3. Transition equation:

$s_{t+1} = s_t - x_t$.

### 4. Problem

The sequence problem is:

$\underset{\{x_t\}_{t\in\mathbb N}}{\max}
\left\{\sum_{t\in\mathbb N} 
\left(\frac{1}{1+r}\right)^tp_t x_t \right\}$

subject to: $s_0 = B$, $s_{t+1} = s_t - x_t$ and $x_t \ge 0$ for all $t$.

The Bellman equation is of the form:

$V(s) = \max_x\left\{px + \frac{1}{1+r} V(s - x)\right\}$

or

$V(s) = \max_x\left\{p(s-s') + \frac{1}{1+r} V(s')\right\}$

### 5. The Euler equation

FOC wrt $s'$: 

$p = \frac{1}{1+r}\frac{dV'(s')}{ds'}$,

and

$\frac{dV'(s')}{ds'} = p' - p'\frac{ds''}{ds'} +
\frac{1}{1+r}\frac{dV''(s'')}{ds''}\frac{ds''}{ds'} = p'$.

Therefore the Euler equation is:

$p = p'\frac{1}{1+r}$.

### 6. The solution

Since the utility function is linear, the solution is piecewise.

If $p_{t+1} = p_t$ for all t, then we sell everything today: $x_0 = B$. This follows from the fact that we get the same absolute payoff from selling a marginal unit in any period, but we discount the future.

If $p_{t+1} > (1+r)p_t$, then $x_t = 0$ for all $t$. 
This is actually a violation of the transversality condition, and so the model is not stationary in this case.

A necessary condition for an interior solution is $p_t(1+r) = p_{t+1}$.


## Exercise 2

### 1. State variables:

Amount of capital $k_t$ and shock to production $z_t$.

### 2. Control variables:

Consumption $c_t$.

### 3. Bellman Equation:

$V(z, k) = \max_c\{U(c) + \beta\mathbb EV(z', k')\}$

subject to:

$k' + c = z k^\alpha + (1-\delta)k$.



#### Import some packages

In [2]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import ar1_approx as ar1

# to print plots inline
%matplotlib inline

### Set Parameters

Parameters:
* $\gamma$ : Coefficient of Relative Risk Aversion
* $\beta$ : Discount factor
* $\delta$ : Rate of depreciation
* $\alpha$ : Curvature of production function
* $\sigma_z$ : Standard dev of productivity shocks


In [3]:
gamma = 0.5
beta = 0.96
delta = 0.05
alpha = 0.4
sigma = 0.2
mu    = 0
rho   = 0

### Create Grid Space

In [4]:
'''
------------------------------------------------------------------------
Create Grid for State Space - Capital    
------------------------------------------------------------------------
lb_k      = scalar, lower bound of capital grid
ub_k      = scalar, upper bound of capital grid 
size_k    = integer, number of grid points in capital state space
k_grid    = vector, size_k x 1 vector of capital grid points 
------------------------------------------------------------------------
'''
lb_k   = 10 # Note that the steady state of k is 11.65 so we create grid around that
ub_k   = 13 
size_k = 30  # Number of grid points for k
k_grid = np.linspace(lb_k, ub_k, size_k)

'''
Create grid of shocks
'''
size_z = 30
ln_z_grid, pi_t = ar1.addacooper(size_z, mu, rho, sigma)
z_grid = np.exp(ln_z_grid)
pi = np.transpose(pi_t)

In [5]:
'''
------------------------------------------------------------------------------
Create grid of current utility values    
------------------------------------------------------------------------------
C        = matrix, current consumption (c=z_tk_t^alpha - k_t+1 + (1-delta)k_t)
U        = matrix, current period utility value for all possible
           choices of w and w' (rows are w, columns w')
------------------------------------------------------------------------------
'''

C = np.zeros((size_k, size_k, size_z)) 
for i in range(size_k): # loop over k
    for j in range(size_k): # loop over k'
        for q in range(size_z):
            C[i, j, q] = z_grid[q] * k_grid[i] **alpha + (1 - delta)*k_grid[i] - k_grid[j]
            # note that if w'>w, consumption negative
# replace 0 and negative consumption with a tiny value 
# This is a way to impose non-negativity on cons

#Not sure if it's good
C[C<=0] = 1e-15
if gamma == 1:
    U = np.log(C)
else:
    U = (C ** (1 - gamma)) / (1 - gamma)
U[C<0] = -9999999

### Value function iteration

In [6]:
'''
------------------------------------------------------------------------
Value Function Iteration    
------------------------------------------------------------------------
VFtol     = scalar, tolerance required for value function to converge
VFdist    = scalar, distance between last two value functions
VFmaxiter = integer, maximum number of iterations for value function
V         = vector, the value functions at each iteration
Vmat      = matrix, the value for each possible combination of w and w'
Vstore    = matrix, stores V at each iteration 
VFiter    = integer, current iteration number
TV        = vector, the value function after applying the Bellman operator
PF        = vector, indicies of choices of w' for all w 
VF        = vector, the "true" value function
------------------------------------------------------------------------
'''

VFtol = 1e-6
VFdist = 7.0 
VFmaxiter = 500 
V = np.zeros((size_k, size_z)) # initial guess at value function
Vmat = np.zeros((size_k, size_k, size_z))
Vstore = np.zeros((size_k, size_k, VFmaxiter)) #initialize Vstore array
VFiter = 1 
while VFdist > VFtol and VFiter < VFmaxiter:
    for i in range(size_k): # loop over epsilon
        for j in range(size_k): # loop over w
            for k in range(size_z):
                EV = 0
                for ii in range(size_z):
                    EV += pi[k, ii]*V[j, ii]
                Vmat[i, j, k] = U[i, j, k] + beta * EV
    Vstore[:, :, VFiter] = V.reshape(size_k, size_z,) # store value function at each iteration for graphing later 
    TV = Vmat.max(1)
    PF = np.argmax(Vmat, axis=1)
    VFdist = (np.absolute(V - TV)).max()  # check distance
    V = TV
    VFiter += 1 

if VFiter < VFmaxiter:
    print('Value function converged after this many iterations:', VFiter)
else:
    print('Value function did not converge')            


VF = V # solution to the functional equation

Value function converged after this many iterations: 367


### Plot Value Function

In [ ]:
# Plot value function
plt.figure()
fig, ax = plt.subplots()
ax.plot(k_grid[5:], VF[5:, 0], label='$z$ = ' + str(round(z_grid[0], 3)))
ax.plot(k_grid[5:], VF[5:, 15], label='$z$ = ' + str(round(z_grid[15], 3)))
ax.plot(k_grid[5:], VF[5:, 30], label='$z$ = ' + str(round(z_grid[30], 3)))
ax.plot(k_grid[5:], VF[5:, 45], label='$z$ = ' + str(round(z_grid[45], 3)))
# Now add the legend with some customizations.
legend = ax.legend(loc='lower right', shadow=False)
# Set the fontsize
for label in legend.get_texts():
    label.set_fontsize('large')
for label in legend.get_lines():
    label.set_linewidth(1.5)  # the legend line width
plt.xlabel('Size of Capital')
plt.ylabel('Value Function')
plt.title('Value Function - neoclassical growth model')
plt.show()


### Plot optimal consumption rule

In [ ]:
# Plot optimal consumption rule as a function of capital size
optK = k_grid[PF]
optC = z_grid * k_grid ** (alpha) + (1 - delta) * k_grid - optK
plt.figure()
fig, ax = plt.subplots()
ax.plot(k_grid[:], optC[:][20], label='Consumption')
# Now add the legend with some customizations.
#legend = ax.legend(loc='upper left', shadow=False)
# Set the fontsize
for label in legend.get_texts():
    label.set_fontsize('large')
for label in legend.get_lines():
    label.set_linewidth(1.5)  # the legend line width
plt.xlabel('Size of Capital')
plt.ylabel('Optimal Consumption')
plt.title('Optimal consumption as size of capital')
plt.show()


In [ ]:
# Plot policy function rule as a function of capital next period
plt.figure()
fig, ax = plt.subplots()
ax.plot(optK[1:][15], optC[1:][15], label='Capital in period t+1')
# Now add the legend with some customizations.
#legend = ax.legend(loc='upper left', shadow=False)
# Set the fontsize
for label in legend.get_texts():
    label.set_fontsize('large')
for label in legend.get_lines():
    label.set_linewidth(1.5)  # the legend line width
plt.xlabel('Size of Capital in next period')
plt.ylabel('Optimal policy')
plt.title('Policy Function, capital next period - growth model')
plt.show()

In [ ]:
k_grid[:]

In [ ]:
optK[1:][11]

In [ ]:
# Plot policy function rule as a function of capital next period
plt.figure()
fig, ax = plt.subplots()
ax.plot(k_grid[:], optK[1:][15], label='Capital in period t+1')
# Now add the legend with some customizations.
#legend = ax.legend(loc='upper left', shadow=False)
# Set the fontsize
for label in legend.get_texts():
    label.set_fontsize('large')
for label in legend.get_lines():
    label.set_linewidth(1.5)  # the legend line width
plt.xlabel('Size of Capital in next period')
plt.ylabel('Optimal policy')
plt.title('Policy Function, capital next period - growth model')
plt.show()

## Question 3

### 1. Bellman equation

$V(z, k) = \max_c\left\{
U(c) + \beta\mathbb E\left[V(z', k)\ |\ z\right]
\right\}$


### Set Parameters

Parameters:
* $\gamma$ : Coefficient of Relative Risk Aversion
* $\beta$ : Discount factor
* $\delta$ : Rate of depreciation
* $\alpha$ : Curvature of production function
* $\mu_z$ : Mean of productivity shock
* $\rho_z$ : AR1 coefficient of productivity shock
* $\sigma_v$ : Standard deviation of v


In [ ]:
#Parameters
gamma = 0.5
beta = 0.96
delta = 0.05
alpha = 0.4
sigmaz = 0.2
mu_z = 0
rho_z = 0.8
sigma_v = 0.1

In [ ]:
'''
------------------------------------------------------------------------
Create Grid for State Space - Shock   
------------------------------------------------------------------------
lb_z      = scalar, lower bound of shock grid
ub_z      = scalar, upper bound of shock grid 
size_z    = integer, number of grid points in shock state space
z_grid    = vector, size_z x 1 vector of shock grid points 
------------------------------------------------------------------------
'''

'''
Create grid of taste shocks
'''
size_z = 30  # Number of grid points
import ar1_approx as ar1
ln_z_grid, pi = ar1.addacooper(size_z, mu_z, rho_z, sigma_v)
z_grid = np.exp(ln_z_grid)
pi_z = np.transpose(pi)


In [ ]:
'''
------------------------------------------------------------------------
Create grid of current utility values    
------------------------------------------------------------------------
C        = matrix, current consumption 
U        = matrix, current period utility value 
------------------------------------------------------------------------
'''

C = np.zeros((size_k, size_k, size_z)) 
for i in range(size_k): # loop over w
    for j in range(size_k): # loop over w'
        for q in range(size_z):
            C[i, j, q] = z_grid[q] * k_grid[i] **alpha + (1 - delta)*k_grid[i] - k_grid[j]
            # note that if w'>w, consumption negative
# replace 0 and negative consumption with a tiny value 
# This is a way to impose non-negativity on cons

#Not sure if it's good
C[C<=0] = 1e-15
if gamma == 1:
    U = np.log(C)
else:
    U = (C ** (1 - gamma)) / (1 - gamma)
U[C<0] = -9999999

In [ ]:
'''
------------------------------------------------------------------------
Value Function Iteration    
------------------------------------------------------------------------
VFtol       = scalar, tolerance required for value function to converge
VFdist      = scalar, distance between last two value functions
VFmaxiter   = integer, maximum number of iterations for value function
V           = matrix, the value functions at each iteration
TV          = matrix, the value function after applying the Bellman operator
PF_discrete = matrix, matrix of policy function: eat=1, not eat=0 
Vstore      = array, stores V at each iteration 
VFiter      = integer, current iteration number
EV          = scalar, expected value function for a given state
VF          = vector, the "true" value function
------------------------------------------------------------------------
'''
VFtol = 1e-6
VFdist = 7.0 
VFmaxiter = 500 
V = np.zeros((size_k, size_z)) # initial guess at value function
Vmat = np.zeros((size_k, size_k, size_z))
Vstore = np.zeros((size_k, size_k, VFmaxiter)) #initialize Vstore array
VFiter = 1 
while VFdist > VFtol and VFiter < VFmaxiter:
    for i in range(size_k): # loop over k
        for j in range(size_k): # loop over k
            for k in range(size_z):
                EV = 0
                for ii in range(size_z):
                    EV += pi_z[k, ii]*V[j, ii]
                Vmat[i, j, k] = U[i, j, k] + beta * EV
    Vstore[:, :, VFiter] = V.reshape(size_k, size_z,) # store value function at each iteration for graphing later 
    TV = Vmat.max(1)
    PF = np.argmax(Vmat, axis=1)
    VFdist = (np.absolute(V - TV)).max()  # check distance
    V = TV
    VFiter += 1 

if VFiter < VFmaxiter:
    print('Value function converged after this many iterations:', VFiter)
else:
    print('Value function did not converge')            


In [ ]:
# Plot value function 
plt.figure()
fig, ax = plt.subplots()
ax.plot(k_grid[1:], VF[1:, 0], label='$z$ = ' + str(round(z_grid[0], 3)))
ax.plot(k_grid[1:], VF[1:, 5], label='$z$ = ' + str(round(z_grid[5], 3)))
ax.plot(k_grid[1:], VF[1:, 15], label='$z$ = ' + str(round(z_grid[15], 3)))
ax.plot(k_grid[1:], VF[1:, 19], label='$z$ = ' + str(round(z_grid[19], 3)))
# Now add the legend with some customizations.
legend = ax.legend(loc='lower right', shadow=False)
# Set the fontsize
for label in legend.get_texts():
    label.set_fontsize('large')
for label in legend.get_lines():
    label.set_linewidth(1.5)  # the legend line width
plt.xlabel('Size of Capital')
plt.ylabel('Value Function')
plt.title('Value Function ')
plt.show()

In [ ]:
#Plot optimal consumption rule
optK = k_grid[PF]
optC = z_grid * k_grid ** (alpha) + (1 - delta) * k_grid - optK
plt.figure()
fig, ax = plt.subplots()
ax.plot(k_grid[:], optC[:][2], label='Consumption' + str(z_grid[2]))
ax.plot(k_grid[:], optC[:][5], label='Consumption' + str(z_grid[5]))
ax.plot(k_grid[:], optC[:][17], label='Consumption' + str(z_grid[17]))
ax.plot(k_grid[:], optC[:][27], label='Consumption' + str(z_grid[27]))
# Now add the legend with some customizations.
#legend = ax.legend(loc='upper left', shadow=False)
# Set the fontsize
for label in legend.get_texts():
    label.set_fontsize('large')
for label in legend.get_lines():
    label.set_linewidth(1.5)  # the legend line width
plt.xlabel('Size of Capital')
plt.ylabel('Optimal Consumption')
plt.title('Capital, consumption')
plt.show()

In [ ]:
#Plot optimal capital next period
optK = k_grid[PF]
plt.figure()
fig, ax = plt.subplots()
ax.plot(k_grid[:], optK[:][2], label='Consumption' + str(z_grid[2]))
ax.plot(k_grid[:], optK[:][5], label='Consumption' + str(z_grid[5]))
ax.plot(k_grid[:], optK[:][17], label='Consumption' + str(z_grid[17]))
ax.plot(k_grid[:], optK[:][27], label='Consumption' + str(z_grid[27]))
# Now add the legend with some customizations.
#legend = ax.legend(loc='upper left', shadow=False)
# Set the fontsize
for label in legend.get_texts():
    label.set_fontsize('large')
for label in legend.get_lines():
    label.set_linewidth(1.5)  # the legend line width
plt.xlabel('Size of Capital')
plt.ylabel('Optimal Capital next period')
plt.title('consumption, consumption next period')
plt.show()

## Question 4

### Bellman equation:
$V(w_t) = \max\left\{ V^U(w_t), V^J(w_t) \right\}$

where:
$V^U(w)=b+\beta EV(w')$

and
$V^J(w)=E_0\sum_{t=0}^\infty\beta^tw=\frac{w}{1-\beta}$

### Set Parameters

Parameters:
* $\gamma$ : Coefficient of Relative Risk Aversion
* $\beta$ : Discount factor
* $\delta$ : Rate of depreciation
* $\alpha$ : Curvature of production function
* $\sigma_z$ : Standard dev of productivity shocks

In [ ]:
beta  = 0.96
b     = np.linspace(0.05,1,0.005)
size_b = len(b)
mu    = 0
sigma = 0.15
rho   = 0

In [ ]:
size_w = 10
'''
Create grid of wage shocks
'''
import ar1_approx as ar1
ln_w_grid, pi_t = ar1.addacooper(size_w, mu, rho, sigma)
w_grid = np.exp(ln_w_grid)
pi = np.transpose(pi_t)
U = w_grid / (1-beta)

In [ ]:
w_grid

In [ ]:
'''
------------------------------------------------------------------------
Value Function Iteration    
------------------------------------------------------------------------
VFtol       = scalar, tolerance required for value function to converge
VFdist      = scalar, distance between last two value functions
VFmaxiter   = integer, maximum number of iterations for value function
V           = matrix, the value functions at each iteration
TV          = matrix, the value function after applying the Bellman operator
PF_discrete = matrix, matrix of policy function: eat=1, not eat=0 
Vstore      = array, stores V at each iteration 
VFiter      = integer, current iteration number
EV          = scalar, expected value function for a given state
U_eat       = matrix, utility from eating cake now
Vwait       = matrix, value of waiting to eat the cake
VF          = vector, the "true" value function
------------------------------------------------------------------------
'''
VFtol = 1e-8 
VFdist = 7.0 
VFmaxiter = 3000 
V = np.zeros(size_w, size_b) # initial guess at value function
TV = np.zeros(size_w, size_b)
PF_discrete = np.zeros(size_w, size_b)
Vstore = np.zeros((size_w, size_b, VFmaxiter)) #initialize Vstore array
VFiter = 1 
while VFdist > VFtol and VFiter < VFmaxiter:
    for i in range(size_w): # loop over w
        EV = 0
        for ii in range(size_w):  # loop over w
            EV += pi[i, ii] * V[ii]   # note can move one space because of how we constructed grid
        U_job = U[i]
        for j in range(size_b):
            Vwait = b[j] + beta * EV 
            TV[i, j] = max(U_job, Vwait)
            PF_discrete[i, j] = U_job >= Vwait  
            
    Vstore[:,:, VFiter] = TV # store value function at each iteration for graphing later 
    VFdist = (np.absolute(V - TV)).max(0)  # check distance
    V = TV
    VFiter += 1 

if VFiter < VFmaxiter:
    print('Value function converged after this many iterations:', VFiter)
else:
    print('Value function did not converge')            


VF = V # solution to the functional equation

In [ ]:
'''
------------------------------------------------------------------------
Find threshold policy functions   
------------------------------------------------------------------------
'''
threshold_w = w_grid[np.argmax(PF_discrete, axis=0)]

In [ ]:
# Plot value function 
plt.figure()
fig, ax = plt.subplots()
ax.plot(w_grid[:], VF[:])
# Now add the legend with some customizations.
# Set the fontsize
for label in legend.get_texts():
    label.set_fontsize('large')
for label in legend.get_lines():
    label.set_linewidth(1.5)  # the legend line width
plt.xlabel('wage')
plt.ylabel('Value Function')
plt.title('Value Function')
plt.show()